In [2]:
import xarray as xr
scn = "2040_2060"
dir_r  = f"/datos/rodr/Datos/WRF/{scn}/"
ds = xr.open_dataset(dir_r + f"{scn}_max.nc")
ds.isel(south_north = 0, west_east = 0).load()

<xarray.Dataset> Size: 215kB
Dimensions:       (XTIME: 7671)
Coordinates:
    XLONG         (XTIME) float32 31kB -100.5 -100.5 -100.5 ... -100.5 -100.5
    XLAT          (XTIME) float32 31kB 18.14 18.14 18.14 ... 18.14 18.14 18.14
  * XTIME         (XTIME) datetime64[ns] 61kB 2040-01-01 ... 2060-12-31
Data variables:
    P_mp          (XTIME) float32 31kB 0.4874 0.7033 0.6962 ... 0.691 0.1337
    hour          (XTIME) float32 31kB 17.0 19.0 19.0 19.0 ... 19.0 19.0 17.0
    perc_morning  (XTIME) float32 31kB 0.4628 0.4631 0.4629 ... 0.4654 1.0

In [3]:
ds.groupby(ds["XTIME"].dt.month).mean()

<xarray.Dataset> Size: 2MB
Dimensions:       (month: 12, south_north: 112, west_east: 132)
Coordinates:
  * month         (month) int64 96B 1 2 3 4 5 6 7 8 9 10 11 12
Dimensions without coordinates: south_north, west_east
Data variables:
    P_mp          (month, south_north, west_east) float32 710kB 0.6736 ... 0....
    hour          (month, south_north, west_east) float32 710kB 18.86 ... 18.38
    perc_morning  (month, south_north, west_east) float32 710kB 0.4566 ... 0....

In [4]:
ds.mean("XTIME")

<xarray.Dataset> Size: 177kB
Dimensions:       (south_north: 112, west_east: 132)
Dimensions without coordinates: south_north, west_east
Data variables:
    P_mp          (south_north, west_east) float32 59kB 0.6054 0.6042 ... 0.5906
    hour          (south_north, west_east) float32 59kB 18.53 18.55 ... 18.5
    perc_morning  (south_north, west_east) float32 59kB 0.5137 0.5136 ... 0.5113

In [ ]:
# une los archivos a lo largo de la variable XTIME
import os
import xarray         as xr
import numpy          as np
import pandas         as pd

# Rutas
dir_o = "/home/rodr/buffalo/rodr/WRF/"
dir_d = f"{dir_o}2040_2060_PV/"
files = os.listdir(dir_d)
files.sort()
f = files[0]

<xarray.Dataset> Size: 258MB
Dimensions:  (XTIME: 2184, south_north: 112, west_east: 132)
Coordinates:
  * XTIME    (XTIME) datetime64[ns] 17kB 2040-01-01 ... 2040-03-31T23:00:00
    XLONG    (south_north, west_east) float32 59kB ...
    XLAT     (south_north, west_east) float32 59kB ...
Dimensions without coordinates: south_north, west_east
Data variables:
    P_mp     (XTIME, south_north, west_east) float64 258MB 0.02274 ... 0.2608
Attributes: (12/135)
    CDI:                             Climate Data Interface version 1.9.5 (ht...
    Conventions:                     CF-1.6
    history:                         Sat Mar 16 09:17:10 2024: cdo mergetime ...
    TITLE:                            OUTPUT FROM WRF V4.3 MODEL
    START_DATE:                      2039-10-29_00:00:00
    SIMULATION_START_DATE:           2039-07-01_00:00:00
    ...                              ...
    ISICE:                           15
    ISURBAN:                         13
    ISOILWATER:                      14
    HYBRID_OPT:                      2
    ETAC:                            0.2
    CDO:                             Climate Data Operators version 1.9.5 (ht...

In [ ]:
# Obtiene los valores máximos diarios
import os
import xarray         as xr
import numpy          as np
import pandas         as pd

dir_d = "/home/rodr/buffalo/rodr/WRF/2040_2060/PV/"
dir_r = "/home/rodr/buffalo/rodr/WRF/2040_2060/max/"
if not os.path.exists(dir_r): os.mkdir(dir_r)

files = os.listdir(dir_d)
files.sort()

for f in files:
    ds = xr.open_dataset(dir_d + f)
    ds["P_mp"] = (ds["P_mp"].astype(np.float64)
        + np.random.randn(*ds["P_mp"].shape) / 1e7)
    var = "P_mp"
    df = ds.resample({"XTIME": "D"}).max().to_dataframe()
    df_2 = ds.to_dataframe()
    df_2["hour"] = df_2.index.get_level_values("XTIME").hour.astype(int)
    df_2 = df_2.reset_index()
    df_2["XTIME"] = df_2["XTIME"].dt.date
    df_2 = df_2.set_index(["XTIME", "south_north", "west_east"])
    df["hour"] = df_2[df_2[var].isin(df[var])]["hour"]
    df["perc_morning"] = (
        ds.shift({"XTIME": -7}).resample({"XTIME": "12h"}).sum()
        / ds.shift({"XTIME": -1}).resample({"XTIME": "D"}).sum(
        ).resample({"XTIME": "12h"}).ffill()).resample({"XTIME": "D"}
        ).first().to_dataframe().sort_index()[var].values
    df.to_xarray().set_coords(["XLONG", "XLAT"]
        ).drop_vars(["south_north", "west_east"]
        ).to_netcdf(dir_r + f)

P_mp       XLONG       XLAT  hour  \
XTIME      south_north west_east                                          
2040-01-01 0           0          0.487367 -100.453476  18.135178    17   
                       1          0.508795 -100.429665  18.135178    20   
                       2          0.495443 -100.405846  18.135178    21   
                       3          0.520208 -100.382027  18.135178    20   
                       4          0.521018 -100.358208  18.135178    20   
...                                    ...         ...        ...   ...   
2040-03-31 111         127        0.748190  -97.428520  20.628967    19   
                       128        0.749387  -97.404701  20.628967    19   
                       129        0.750683  -97.380882  20.628967    19   
                       130        0.752021  -97.357063  20.628967    19   
                       131        0.752962  -97.333252  20.628967    19   

                                  perc_morning  
XTIME      south_north west_east                
2040-01-01 0           0              0.462845  
                       1              0.459005  
                       2              0.461756  
                       3              0.458114  
                       4              0.456628  
...                                        ...  
2040-03-31 111         127            0.507997  
                       128            0.507328  
                       129            0.499948  
                       130            0.502587  
                       131            0.507142  

[1345344 rows x 5 columns]

In [ ]:
import dask.dataframe as dd
from dask.diagnostics import ProgressBar
df = ds.resample({"XTIME": "D"}).max().to_dataframe(
    ).drop(columns = ["XLONG", "XLAT"])
df = df.reset_index()
df["XTIME"] = df["XTIME"].astype(str)
df_2 = ds.to_dataframe()
df_2["hour"] = df_2.index.get_level_values("XTIME").hour.astype(int)
df_2 = df_2.reset_index()
df_2["XTIME"] = df_2["XTIME"].dt.date.astype(str)
df_2 = df_2.set_index(["XTIME", "south_north", "west_east"])
df_i = dd.from_pandas(df.iloc[:132*112*10], chunksize=66)
df_i["hour"] = df_i.apply(lambda x: df_2[df_2["P_mp"]==x["P_mp"]].loc[
    (x["XTIME"], x["south_north"], x["west_east"]), "hour"],
    axis = 1, meta = pd.Series(dtype = str))
with ProgressBar(): df_i.compute()

[########################################] | 100% Completed | 502.43 s
